# Modelo Regresión Logistica GS

In [ ]:
# Contar los valores nulos por columna para comprobar que no haya nulos
print(df[features_to_scale + ["Favorite_Wins"]].isna().sum())



In [ ]:
X = df[features_to_scale]  # Asegurarse de que no haya columnas extra
y = df["Favorite_Wins"]


In [ ]:
#Comprobamos que no haya nulos en x ni en y
print("¿NaNs en X?:", X.isna().any().any()) 
print("¿NaNs en y?:", y.isna().any())


In [ ]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de regresión logística
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Calcular y mostrar la métrica
score = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy:", round(score, 3))

Que el resultado de Balanced Accuracy = 0.545 indica que el modelo está rindiendo solo ligeramente mejor que un clasificador aleatorio (que tendría una balanced accuracy de ~0.5 en un problema balanceado). Esto puede deberse a que las clases esten desbalanceadas 

verificamos cuántas veces gana el favorito (Favorite_Wins). Si hay muchas más 1s que 0s, la métrica accuracy o incluso balanced_accuracy puede estar sesgada.

In [ ]:
print(y.value_counts(normalize=True))


El favorito gana en el 71.3% de los casos (Favorite_Wins = 1)  

El no favorito gana solo el 28.6% de las veces (Favorite_Wins = 0)  

dataset está desbalanceado, y eso afecta directamente al rendimiento de modelos como regresión logística, que no manejan bien el desbalance sin ajustes. Un clasificador que siempre predice 1 tendría una accuracy del 71.3%, pero no estaría aprendiendo nada útil.


Esto hace que el modelo penalice más los errores en la clase minoritaria (0), equilibrando el aprendizaje:

Cuando usas un modelo como LogisticRegression sin el argumento class_weight='balanced', trata a todas las clases por igual, aunque tengas muchas más observaciones de una clase que de otra.

Sin embargo, cuando los datos están desbalanceados (por ejemplo, 71.3% de favoritos ganan y solo 28.6% pierden), el modelo puede aprender a "ir siempre a lo seguro" y predecir siempre que el favorito gana. Esto te da una aparente alta accuracy, pero no está aprendiendo realmente.

Sin balanced: accuracy puede ser buena, pero solo porque acierta muchos “gana el favorito”.

Con balanced: el modelo acierta más veces cuando el favorito pierde, lo cual es mucho más difícil y valioso.


In [ ]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de regresión logística con ajuste por desbalance
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluar con balanced accuracy
score = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy:", round(score, 3))

### Prueba con validación cruzada

In [ ]:
# Usamos solo las columnas que queremos escalar
X = df[features_to_scale]
y = df["Favorite_Wins"]  

# Modelo con validación cruzada (5 folds)
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
y_pred_cv = cross_val_predict(model, X, y, cv=5)

# Evaluación
score_cv = balanced_accuracy_score(y, y_pred_cv)
print("Balanced Accuracy (CV):", round(score_cv, 3))

## Evaluación del modelo 

In [ ]:
# Predicciones con validación cruzada
y_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')
y_pred = (y_proba[:, 1] >= 0.5).astype(int)  # convertir probabilidades en clases

# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))

## Conclusiones

###  **Métricas del Modelo**

| Métrica               | Valor | ¿Qué significa?                                                                                                            |
|-----------------------|-------|----------------------------------------------------------------------------------------------------------------------------|
| **Accuracy**          | 0.642 | El modelo acierta en el 64,2% de los casos totales. Puede estar sesgado si las clases están desbalanceadas.               |
| **Balanced Accuracy** | 0.634 | La media de los aciertos por clase. Corrige el sesgo de clases desbalanceadas. Más fiable que accuracy en este caso.      |
| **Precision**         | 0.809 | Cuando el modelo predice "gana el favorito", acierta el 80,9% de las veces.                                                |
| **Recall**            | 0.652 | De todas las veces que el favorito gana, el modelo lo detecta el 65,2%.                                                    |
| **F1-Score**          | 0.722 | Media armónica entre precisión y recall. Mide el equilibrio entre ambos.                                                   |
| **AUC-ROC**           | 0.684 | Capacidad del modelo para distinguir entre ganadores y perdedores, usando probabilidades. Entre 0.5 (azar) y 1 (perfecto). |
| **Log Loss**          | 0.643 | Penaliza errores de predicción probabilística. Cuanto más bajo, mejor.                                                     |

---

###  **Matriz de Confusión**

|                         | Predicho: 0 | Predicho: 1 | Total Real |
|-------------------------|-------------|-------------|------------|
| **Real: 0** (pierde)    | 2264        | 1407        | 3671       |
| **Real: 1** (gana)      | 3187        | 5969        | 9156       |

-  **2264 veces** el modelo acertó al predecir que el favorito **no ganaba** (derrota).
-  **5969 veces** acertó al predecir que el favorito **ganaba**.
-  **1407 veces** predijo "gana" cuando perdió.
-  **3187 veces** predijo "pierde" cuando ganó.

---

###  **Classification Report**

| Clase | Precision | Recall | F1-Score | Soporte | Interpretación |
|-------|-----------|--------|----------|---------|----------------|
| **0** (pierde el favorito) | 0.42 | 0.62 | 0.50 | 3671 | Buena capacidad para detectar derrotas. Aunque la precisión es baja, el recall es decente (detecta 6 de cada 10). |
| **1** (gana el favorito)   | 0.81 | 0.65 | 0.72 | 9156 | Alta precisión (cuando predice "gana", suele acertar), aunque se le escapan algunas victorias reales. |


 ## Random Forest + validación cruzada GS

In [ ]:
# Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Validación cruzada
y_proba_rf = cross_val_predict(rf_model, X, y, cv=5, method='predict_proba')
y_pred_rf = (y_proba_rf[:, 1] >= 0.5).astype(int)

In [ ]:

# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_rf), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_rf), 3))
print("Precision:", round(precision_score(y, y_pred_rf), 3))
print("Recall:", round(recall_score(y, y_pred_rf), 3))
print("F1-Score:", round(f1_score(y, y_pred_rf), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_rf[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_rf[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_rf))
print("\nClassification Report:\n", classification_report(y, y_pred_rf))

###  Comparativa de Modelos: Regresión Logística vs Random Forest

| Métrica               | Regresión Logística | Random Forest | Comentario |
|-----------------------|---------------------|----------------|------------|
| **Accuracy**          | 0.642               | 0.707          | RF tiene mejor rendimiento general. |
| **Balanced Accuracy** | 0.634               | 0.554          | Logística más equilibrada entre clases. |
| **Precision**         | 0.809               | 0.739          | Logística es más precisa al predecir victorias. |
| **Recall**            | 0.652               | 0.912          | RF detecta muchas más victorias del favorito. |
| **F1-Score**          | 0.722               | 0.816          | RF logra mejor equilibrio entre precisión y recall. |
| **AUC-ROC**           | 0.684               | 0.673          | Muy similares; capacidad discriminativa comparable. |
| **Log Loss**          | 0.643               | 0.571          | RF produce mejores probabilidades calibradas. |

---

###  Classification Report del Modelo (Random Forest)

| Clase | Precision | Recall | F1-Score | Soporte | Interpretación |
|-------|-----------|--------|----------|---------|----------------|
| **0** (pierde el favorito) | 0.47 | 0.20 | 0.28 | 3671 | Predice derrotas con precisión moderada, pero **solo detecta el 20%** de las reales. Bajo recall. |
| **1** (gana el favorito)   | 0.74 | 0.91 | 0.82 | 9156 | Excelente rendimiento: **detecta el 91%** de las victorias y con buena precisión. |

---

###  Conclusiones:

-  **Random Forest** mejora el rendimiento general, sobre todo en la **detección de victorias del favorito** (recall 91,2%).
- Sin embargo, **sacrifica rendimiento en la clase minoritaria** (derrotas), con un recall del 20% y balanced accuracy inferior.
-  La **regresión logística** es más equilibrada entre clases, pero menos potente globalmente.



## Modelo Gradient Boosting GS

In [ ]:
# Modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Validación cruzada
y_proba_gb = cross_val_predict(gb_model, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)

In [ ]:
# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))

###  Comparativa de Modelos: Regresión Logística vs Random Forest vs Gradient Boosting

| Métrica               | Regresión Logística | Random Forest | Gradient Boosting | Comentario |
|-----------------------|---------------------|----------------|-------------------|------------|
| **Accuracy**          | 0.642               | 0.707          | 0.717             | GB y RF superan claramente a Logística. |
| **Balanced Accuracy** | 0.634               | 0.554          | 0.553             | Logística es la más equilibrada entre clases. |
| **Precision**         | 0.809               | 0.739          | 0.738             | Logística es más precisa al predecir victorias. |
| **Recall**            | 0.652               | 0.912          | 0.936             | GB detecta la mayor parte de las victorias. |
| **F1-Score**          | 0.722               | 0.816          | 0.825             | GB logra el mejor equilibrio entre precisión y recall. |
| **AUC-ROC**           | 0.684               | 0.673          | 0.699             | GB tiene la mejor capacidad discriminativa. |
| **Log Loss**          | 0.643               | 0.571          | 0.547             | GB también ofrece las mejores probabilidades calibradas. |

---

###  Classification Report del Modelo (Gradient Boosting)

| Clase | Precision | Recall | F1-Score | Soporte | Interpretación |
|-------|-----------|--------|----------|---------|----------------|
| **0** (pierde el favorito) | 0.51 | 0.17 | 0.26 | 3671 | Muy baja capacidad para detectar derrotas. Solo identifica el 17% de ellas, con precisión limitada. |
| **1** (gana el favorito)   | 0.74 | 0.94 | 0.83 | 9156 | Excelente rendimiento: detecta el 94% de las victorias con buena precisión. |

---

###  Conclusiones Generales

- **Regresión Logística**: Modelo más equilibrado entre clases, pero con menor rendimiento global.
- **Random Forest**: Muy buen desempeño para victorias del favorito, aunque limitado en la detección de derrotas.
- **Gradient Boosting**: Mejor modelo en métricas globales (F1, AUC, Log Loss), pero **sacrifica notablemente la clase minoritaria** (recall del 17% en derrotas).
- Si el objetivo es **predecir victorias del favorito con alta fiabilidad**, **Gradient Boosting es el modelo más eficaz**.



## Modelo Gradient Boosting balanceado GS

In [ ]:
gb_model = HistGradientBoostingClassifier(class_weight='balanced', max_iter=100, learning_rate=0.1, max_depth=6, random_state=42)

# Validación cruzada
y_proba_gb = cross_val_predict(gb_model, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)

In [ ]:
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

print("Confusion Matrix:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))

### Resultados del Modelo: HistGradientBoostingClassifier con `class_weight='balanced'`

| Métrica               | Valor  | Interpretación |
|-----------------------|--------|----------------|
| **Accuracy**          | 0.633  | Acierta en el 63,3% de los casos totales. Más equilibrado pero algo menos preciso en global. |
| **Balanced Accuracy** | 0.646  | La mejor hasta ahora. Buen equilibrio entre clases, útil si interesan las derrotas del favorito. |
| **Precision**         | 0.825  | Cuando predice que gana el favorito, acierta el 82,5% de las veces. Muy alta precisión. |
| **Recall**            | 0.617  | Detecta el 61,7% de las victorias reales del favorito. |
| **F1-Score**          | 0.706  | Buen equilibrio entre precisión y recall. Muy similar a regresión logística. |
| **AUC-ROC**           | 0.695  | Alta capacidad para distinguir entre clases en base a probabilidades. |
| **Log Loss**          | 0.618  | Mejores predicciones probabilísticas que la regresión logística, peor que Gradient Boosting sin balancear. |

---

###  Matriz de Confusión

|                 | Predicho: 0 | Predicho: 1 |
|-----------------|-------------|-------------|
| **Real: 0**     | 2475        | 1196        |
| **Real: 1**     | 3506        | 5650        |

-  Acierta **2475** derrotas del favorito (mucho mejor que otros modelos).
-  Acierta **5650** victorias del favorito.
-  Falla **1196** veces diciendo que ganaría y no fue así.
-  Falla **3506** veces diciendo que perdería y en realidad ganó.

---

###  Classification Report

| Clase | Precision | Recall | F1-Score | Soporte | Interpretación |
|-------|-----------|--------|----------|---------|----------------|
| **0** (pierde el favorito) | 0.41 | 0.67 | 0.51 | 3671 | Mejora notable en detección de derrotas (67%). Aunque la precisión no es alta, acierta muchas de las derrotas reales. |
| **1** (gana el favorito)   | 0.83 | 0.62 | 0.71 | 9156 | Muy buena precisión (83%). Aunque se le escapan algunas victorias, mantiene buen balance. |

---

###  Conclusión

Este modelo logra el **mejor equilibrio entre clases** hasta el momento. Es ideal si te interesa **detectar sorpresas (derrotas del favorito)** sin sacrificar completamente el rendimiento global. Aunque pierde algo de recall en victorias, gana mucho en representar mejor ambas clases.



## MLPClassifier GS 

In [ ]:
# Modelo MLP
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,),  # una capa oculta con 100 neuronas
    max_iter=300,
    alpha=1e-4,
    solver='adam',
    random_state=42
)

# Validación cruzada
y_proba_mlp = cross_val_predict(mlp_model, X, y, cv=5, method='predict_proba')
y_pred_mlp = (y_proba_mlp[:, 1] >= 0.5).astype(int)


In [ ]:
# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_mlp), 3))
print("Precision:", round(precision_score(y, y_pred_mlp), 3))
print("Recall:", round(recall_score(y, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_mlp[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y, y_pred_mlp))

### Resultados del Modelo: MLPClassifier (Perceptrón Multicapa)

| Métrica               | Valor  | Interpretación |
|-----------------------|--------|----------------|
| **Accuracy**          | 0.714  | Buen rendimiento general: acierta en el 71,4% de los casos. |
| **Balanced Accuracy** | 0.556  | Bastante baja: el modelo favorece fuertemente la clase 1 (victorias del favorito). |
| **Precision**         | 0.739  | Cuando predice que gana el favorito, acierta el 73,9% de las veces. |
| **Recall**            | 0.925  | Excelente: detecta el 92,5% de las victorias reales. |
| **F1-Score**          | 0.822  | Excelente equilibrio entre precisión y recall para la clase mayoritaria. |
| **AUC-ROC**           | 0.699  | Muy buen poder discriminativo usando probabilidades. |
| **Log Loss**          | 0.551  | Predicciones probabilísticas bastante buenas. |

---

###  Matriz de Confusión

|                 | Predicho: 0 | Predicho: 1 |
|-----------------|-------------|-------------|
| **Real: 0**     | 688         | 2983        |
| **Real: 1**     | 689         | 8467        |

-  El modelo acierta **8467 victorias del favorito** (recall altísimo).
-  Solo detecta **688 derrotas reales**, y se equivoca **2983 veces** prediciendo que ganaría cuando en realidad perdió.

---

###  Classification Report (Resumen)

| Clase | Precision | Recall | F1-Score | Soporte |
|-------|-----------|--------|----------|---------|
| **0** (pierde el favorito) | 0.50 | 0.19 | 0.27 | 3671 |
| **1** (gana el favorito)   | 0.74 | 0.92 | 0.82 | 9156 |

---

###  Conclusión

El MLP muestra muy buen rendimiento al predecir victorias del favorito**, con alta precisión y recall, pero ignora en gran parte las derrotas (clase minoritaria).


 


# MLPClassifier con SMOTE GS
Cuando entrenas un modelo con clases desbalanceadas, este suele ignorar la clase minoritaria (como las derrotas del favorito), porque aprende a predecir siempre la mayoritaria para maximizar el acierto global.
SMOTE lo soluciona generando datos sintéticos (falsos pero realistas) de la clase minoritaria, haciendo que el modelo:
- preste atención a esa clase,
- aprenda patrones también cuando el favorito pierde.

In [ ]:
# 1. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# 3. Entrenar MLP sobre datos balanceados
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_model.fit(X_train_sm, y_train_sm)

# 4. Predecir sobre el test original
y_pred_mlp = mlp_model.predict(X_test)
y_proba_mlp = mlp_model.predict_proba(X_test)


In [ ]:
# 5. Métricas
print("Accuracy:", round(accuracy_score(y_test, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y_test, y_pred_mlp), 3))
print("Precision:", round(precision_score(y_test, y_pred_mlp), 3))
print("Recall:", round(recall_score(y_test, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y_test, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y_test, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y_test, y_proba_mlp[:, 1]), 3))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y_test, y_pred_mlp))

###  Resultados del Modelo: MLPClassifier con SMOTE

| Métrica               | Valor  | Interpretación |
|-----------------------|--------|----------------|
| **Accuracy**          | 0.625  | Acierta en el 62,5% de los casos. Disminuye ligeramente frente al MLP sin balancear, pero mejora la detección de derrotas. |
| **Balanced Accuracy** | 0.632  | Mucho mejor que el MLP sin balancear (0.556). Buen avance en el tratamiento de clases desbalanceadas. |
| **Precision**         | 0.815  | Muy alta cuando predice victorias (clase 1). |
| **Recall**            | 0.615  | Más bajo que antes en victorias, pero mejora mucho en derrotas. |
| **F1-Score**          | 0.701  | Buen equilibrio entre precisión y recall. |
| **AUC-ROC**           | 0.681  | Alta capacidad discriminativa usando probabilidades. |
| **Log Loss**          | 0.638  | Predicciones probabilísticas razonablemente fiables. |

---

###  Matriz de Confusión

|                 | Predicho: 0 | Predicho: 1 |
|-----------------|-------------|-------------|
| **Real: 0**     | 474         | 256         |
| **Real: 1**     | 707         | 1129        |

-  Detecta correctamente **474 derrotas del favorito** (recall clase 0: **65%**).
-  Detecta correctamente **1129 victorias del favorito** (recall clase 1: **61%**).
-  Se confunde más que antes, pero **recupera la clase minoritaria**, lo que muchos modelos anteriores no lograban.

---

###  Classification Report (Resumen)

| Clase | Precision | Recall | F1-Score | Soporte |
|-------|-----------|--------|----------|---------|
| **0** (pierde el favorito) | 0.40 | 0.65 | 0.50 | 730 |
| **1** (gana el favorito)   | 0.82 | 0.61 | 0.70 | 1836 |

---

###  Conclusión

El MLP entrenado con **SMOTE** logra un balance mucho mejor entre clases, especialmente en detectar derrotas del favorito.

-  Aunque su accuracy baja un poco, es uno de los mejores modelos para detectar sorpresas (cuando el favorito pierde).
-  El coste es una ligera pérdida de precisión general, pero se compensa con una visión más equilibrada del problema.



# Comparación de modelos

###  Comparativa de Modelos: Rendimiento Global

| Modelo                    | Accuracy | Balanced Acc | Precision | Recall | F1-Score | AUC-ROC | Log Loss | Comentario principal |
|---------------------------|----------|---------------|-----------|--------|----------|---------|----------|------------------------|
| **Regresión Logística**   | 0.642    | 0.634         | 0.809     | 0.652  | 0.722    | 0.684   | 0.643    | Buen equilibrio, sencillo y fiable |
| **Random Forest**         | 0.707    | 0.554         | 0.739     | 0.912  | 0.816    | 0.673   | 0.571    | Muy sesgado hacia victorias |
| **Gradient Boosting**     | 0.717    | 0.553         | 0.738     | 0.936  | 0.825    | 0.699   | 0.547    | Mejor F1, pero ignora derrotas |
| **GB con Balanceo**       | 0.633    | 0.646         | 0.825     | 0.617  | 0.706    | 0.695   | 0.618    | Mejor balance entre clases |
| **MLP (sin balancear)**   | 0.714    | 0.556         | 0.739     | 0.925  | 0.822    | 0.699   | 0.551    | Gran recall en victorias, malo en derrotas |
| **MLP + SMOTE**           | 0.625    | 0.632         | 0.815     | 0.615  | 0.701    | 0.681   | 0.638    | Buen equilibrio, mejor para detectar derrotas |


**-  Mejor modelo balanceado:** GB con balanceo o MLP + SMOTE, si  interesa detectar **derrotas del favorito**. Ambos mejoran considerablemente la **Balanced Accuracy** y el **recall de la clase minoritaria**, a costa de algo de rendimiento global.

**-Mejor en rendimiento general (F1 más alto):** Gradient Boosting sin balanceo, con un **F1-Score de 0.825** y **recall del 93,6%** para las victorias. Ideal si solo interesa acertar al máximo los casos comunes.

**- Mejor AUC-ROC (discriminación probabilística):** Gradient Boosting sin balanceo (**0.699**) y MLP sin balancear (**0.699**) empatan como los más fiables en cuanto a la calidad de las probabilidades que asignan.


# Favorite_Loses

In [ ]:
X = df[features_to_scale]  # Asegurarse de que no haya columnas extra
y = df["Favorite_Loses"]


In [ ]:
#Comprobamos que no haya nulos en x ni en y
print("¿NaNs en X?:", X.isna().any().any()) 
print("¿NaNs en y?:", y.isna().any())


In [ ]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de regresión logística
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Calcular y mostrar la métrica
score = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy:", round(score, 3))

In [ ]:
print(y.value_counts(normalize=True))

In [ ]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de regresión logística con ajuste por desbalance
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluar con balanced accuracy
score = balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy:", round(score, 3))

## Prueba con validación cruzada

In [ ]:
# Usamos solo las columnas que queremos escalar
X = df[features_to_scale]
y = df["Favorite_Loses"]  

# Modelo con validación cruzada (5 folds)
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
y_pred_cv = cross_val_predict(model, X, y, cv=5)

# Evaluación
score_cv = balanced_accuracy_score(y, y_pred_cv)
print("Balanced Accuracy (CV):", round(score_cv, 3))

## Evaluacion del modelo

In [ ]:
# Predicciones con validación cruzada
y_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')
y_pred = (y_proba[:, 1] >= 0.5).astype(int)  # convertir probabilidades en clases

# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))

 ## Random Forest + validación cruzada

In [ ]:
# Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Validación cruzada
y_proba_rf = cross_val_predict(rf_model, X, y, cv=5, method='predict_proba')
y_pred_rf = (y_proba_rf[:, 1] >= 0.5).astype(int)


In [ ]:
# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_rf), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_rf), 3))
print("Precision:", round(precision_score(y, y_pred_rf), 3))
print("Recall:", round(recall_score(y, y_pred_rf), 3))
print("F1-Score:", round(f1_score(y, y_pred_rf), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_rf[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_rf[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_rf))
print("\nClassification Report:\n", classification_report(y, y_pred_rf))

In [ ]:
# Modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Validación cruzada
y_proba_gb = cross_val_predict(gb_model, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)


In [ ]:
# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))


## Modelo Gradient Boosting balanceado

In [ ]:
gb_model = HistGradientBoostingClassifier(class_weight='balanced', max_iter=100, learning_rate=0.1, max_depth=6, random_state=42)

# Validación cruzada
y_proba_gb = cross_val_predict(gb_model, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)



In [ ]:
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

print("Confusion Matrix:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))


## MLPClassifier


In [ ]:



# Modelo MLP
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,),  # una capa oculta con 100 neuronas
    max_iter=300,
    alpha=1e-4,
    solver='adam',
    random_state=42
)

# Validación cruzada
y_proba_mlp = cross_val_predict(mlp_model, X, y, cv=5, method='predict_proba')
y_pred_mlp = (y_proba_mlp[:, 1] >= 0.5).astype(int)


In [ ]:
# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_mlp), 3))
print("Precision:", round(precision_score(y, y_pred_mlp), 3))
print("Recall:", round(recall_score(y, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_mlp[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y, y_pred_mlp))

In [ ]:


# MLPClassifier con SMOTE

# 1. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# 3. Entrenar MLP sobre datos balanceados
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_model.fit(X_train_sm, y_train_sm)

# 4. Predecir sobre el test original
y_pred_mlp = mlp_model.predict(X_test)
y_proba_mlp = mlp_model.predict_proba(X_test)


In [ ]:
# 5. Métricas
print("Accuracy:", round(accuracy_score(y_test, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y_test, y_pred_mlp), 3))
print("Precision:", round(precision_score(y_test, y_pred_mlp), 3))
print("Recall:", round(recall_score(y_test, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y_test, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y_test, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y_test, y_proba_mlp[:, 1]), 3))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y_test, y_pred_mlp))


##  Plan de Mejora de Modelos

### Seleccionar los algoritmos más prometedores
- Criterio de selección: **mayor AUC-ROC**
- Modelos seleccionados:
  - `Gradient Boosting` (sin balanceo)
  - `MLPClassifier (sin balancear)`

---

### Optimización de hiperparámetros

####  Objetivo:
- Mejorar el rendimiento de los modelos elegidos sin modificar su estructura interna.

####  Métodos que usaré:
- `RandomizedSearchCV`: ideal para pruebas rápidas con espacio de búsqueda definido.
- `Optuna`: optimización bayesiana, más inteligente y eficiente en exploración de hiperparámetros.

#### Parámetros a optimizar:

**Gradient Boosting:**
- `learning_rate`, `max_depth`, `n_estimators`, `subsample`

**MLPClassifier:**
- `hidden_layer_sizes`, `alpha`, `learning_rate_init`, `solver`, `activation`

---


In [ ]:
X = df[features_to_scale]  # Asegurarse de que no haya columnas extra
y = df["Favorite_Wins"]

# Modelo Gradient Boosting

In [ ]:
# Modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Validación cruzada
y_proba_gb = cross_val_predict(gb_model, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)


In [ ]:

# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))

# Modelo MLP

In [ ]:
# Modelo MLP
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,),  # una capa oculta con 100 neuronas
    max_iter=300,
    alpha=1e-4,
    solver='adam',
    random_state=42
)

# Validación cruzada
y_proba_mlp = cross_val_predict(mlp_model, X, y, cv=5, method='predict_proba')
y_pred_mlp = (y_proba_mlp[:, 1] >= 0.5).astype(int)



In [ ]:

# Métricas
print("Accuracy:", round(accuracy_score(y, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_mlp), 3))
print("Precision:", round(precision_score(y, y_pred_mlp), 3))
print("Recall:", round(recall_score(y, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_mlp[:, 1]), 3))

# Matriz de confusión e informe
print("Confusion Matrix:\n", confusion_matrix(y, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y, y_pred_mlp))


## Mejora Gradient Boosting

In [ ]:
# 1. Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0]
}

# 2. Instanciar el modelo base
gb_model = GradientBoostingClassifier(random_state=42)

# 3. Búsqueda aleatoria con validación cruzada
random_search = RandomizedSearchCV(
    gb_model, param_dist, n_iter=20, cv=5,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
random_search.fit(X, y)

# 4. Mejor modelo encontrado
best_gb = random_search.best_estimator_
print("Mejores hiperparámetros encontrados:", random_search.best_params_)

# 5. Validación cruzada con el mejor modelo
y_proba_gb = cross_val_predict(best_gb, X, y, cv=5, method='predict_proba')
y_pred_gb = (y_proba_gb[:, 1] >= 0.5).astype(int)

# 6. Evaluación
print("\nMétricas:")
print("Accuracy:", round(accuracy_score(y, y_pred_gb), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_gb), 3))
print("Precision:", round(precision_score(y, y_pred_gb), 3))
print("Recall:", round(recall_score(y, y_pred_gb), 3))
print("F1-Score:", round(f1_score(y, y_pred_gb), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_gb[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_gb[:, 1]), 3))

print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred_gb))
print("\nClassification Report:\n", classification_report(y, y_pred_gb))

In [ ]:
# 1. Instanciar modelo con mejores hiperparámetros
model = GradientBoostingClassifier(
    subsample=0.6,
    n_estimators=200,
    max_depth=5,
    learning_rate=0.01,
    random_state=42
)

# 2. Validación cruzada con probabilidad
y_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba', n_jobs=-1)
y_pred = (y_proba[:, 1] >= 0.5).astype(int)

# 3. Métricas
print("MÉTRICAS DEL MODELO OPTIMIZADO (Gradient Boosting):")
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba[:, 1]), 3))

# 4. Matriz de Confusión
print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred))

# 5. Reporte de Clasificación
print("\nClassification Report:\n", classification_report(y, y_pred))


In [ ]:
# 1. Inicializar predicciones vacías
y_proba = np.zeros((len(y), 2))
y_pred = np.zeros(len(y))

# 2. Validación cruzada manual con sample_weight
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in kf.split(X, y):
    # Usamos .iloc para indexar por posición en Pandas
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Calcular pesos balanceados solo para entrenamiento
    weights = compute_sample_weight(class_weight='balanced', y=y_train)

    # Modelo Gradient Boosting optimizado
    model = GradientBoostingClassifier(
        subsample=0.6,
        n_estimators=200,
        max_depth=5,
        learning_rate=0.01,
        random_state=42
    )

    # Entrenar con pesos
    model.fit(X_train, y_train, sample_weight=weights)

    # Guardar predicciones
    y_proba[test_idx] = model.predict_proba(X_test)
    y_pred[test_idx] = model.predict(X_test)

# 3. Evaluación
print("MÉTRICAS CON sample_weight='balanced':")
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba[:, 1]), 3))

print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred.astype(int)))
print("\nClassification Report:\n", classification_report(y, y_pred.astype(int)))


## Optuna 

In [ ]:
# Objetivo de optimización
def objective(trial):
    # Hiperparámetros a optimizar
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    }

    # Validación cruzada
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        weights = compute_sample_weight(class_weight='balanced', y=y_train)

        model = GradientBoostingClassifier(
            **params,
            random_state=42
        )

        model.fit(X_train, y_train, sample_weight=weights)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        aucs.append(auc)

    return np.mean(aucs)

# Crear y ejecutar estudio
study = optuna.create_study(direction='maximize', study_name='GB_AUC_Optimization')
study.optimize(objective, n_trials=100, n_jobs=1)  # subir n_trials a 50-100 si quieres mejor resultado

# Mostrar mejor resultado
print(" Mejor AUC-ROC:", round(study.best_value, 4))
print(" Mejores hiperparámetros encontrados:")
print(study.best_params)

In [ ]:
# Crear modelo con los mejores hiperparámetros encontrados
final_model = GradientBoostingClassifier(
    learning_rate=0.0222992728201512,
    n_estimators=484,
    max_depth=3,
    subsample=0.6374765489547842,
    random_state=42
)

# Calcular pesos para clases desbalanceadas
weights = compute_sample_weight(class_weight='balanced', y=y)

# Entrenar con todos los datos
final_model.fit(X, y, sample_weight=weights)


In [ ]:
# Inicializar
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_proba = np.zeros(len(y))
y_pred = np.zeros(len(y))

# Validación cruzada manual
for train_idx, test_idx in kf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    w = compute_sample_weight(class_weight='balanced', y=y_train)

    model = GradientBoostingClassifier(
        learning_rate=0.0222992728201512,
        n_estimators=484,
        max_depth=3,
        subsample=0.6374765489547842,
        random_state=42
    )

    model.fit(X_train, y_train, sample_weight=w)
    y_proba[test_idx] = model.predict_proba(X_test)[:, 1]
    y_pred[test_idx] = model.predict(X_test)

# Evaluación
y_pred = y_pred.astype(int)
print("MÉTRICAS DEL MODELO FINAL (Optuna ):")
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba), 3))
print("Log Loss:", round(log_loss(y, y_proba), 3))
print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))


### Mejora MLP

In [ ]:
# Lista de columnas a escalar
features_to_scale = [
    "Surface_WinRate_Favorite", "Surface_WinRate_Not_Favorite",
    "Surface_Matches_Favorite", "Surface_Matches_Not_Favorite",
    "WinStreak_Favorite", "WinStreak_Not_Favorite", "Win_Streak_Diff",
    "Rank_Favorite", "Rank_Not_Favorite", "Rank_Diff_Signed", "Rank_Diff_Abs",
    "GrandSlams_Favorite", "GrandSlams_Not_Favorite"
]

# Asegurarse de que son numéricos
for col in features_to_scale:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Escalado con StandardScaler
scaler = StandardScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

In [ ]:
X = df[features_to_scale]  # Asegurarse de que no haya columnas extra
y = df["Favorite_Wins"]


In [ ]:
# 1. Espacio de búsqueda de hiperparámetros
param_dist_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2],
    'learning_rate_init': [0.0005, 0.001, 0.01, 0.1],
    'solver': ['adam', 'sgd'],
    'activation': ['relu', 'tanh']
}

# 2. Modelo base
mlp_model = MLPClassifier(max_iter=300, random_state=42)

# 3. Búsqueda aleatoria con validación cruzada
mlp_random_search = RandomizedSearchCV(
    mlp_model,
    param_distributions=param_dist_mlp,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42
)

mlp_random_search.fit(X, y)

# 4. Mejor modelo encontrado
best_mlp = mlp_random_search.best_estimator_
print("Mejores hiperparámetros encontrados:", mlp_random_search.best_params_)

# 5. Validación cruzada con el mejor modelo
y_proba_mlp = cross_val_predict(best_mlp, X, y, cv=5, method='predict_proba')
y_pred_mlp = (y_proba_mlp[:, 1] >= 0.5).astype(int)

# 6. Evaluación
print("\n MÉTRICAS MLPClassifier (RandomizedSearchCV):")
print("Accuracy:", round(accuracy_score(y, y_pred_mlp), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred_mlp), 3))
print("Precision:", round(precision_score(y, y_pred_mlp), 3))
print("Recall:", round(recall_score(y, y_pred_mlp), 3))
print("F1-Score:", round(f1_score(y, y_pred_mlp), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba_mlp[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba_mlp[:, 1]), 3))

print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred_mlp))
print("\nClassification Report:\n", classification_report(y, y_pred_mlp))

In [ ]:
# Mostrar progreso en consola
#optuna.logging.set_verbosity(optuna.logging.INFO)

# Objetivo de optimización
def objective(trial):
    params = {
        'hidden_layer_sizes': trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50, 50), (100, 50), (100, 100)]),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e-2, log=True),
        'learning_rate_init': trial.suggest_float('learning_rate_init', 0.0005, 0.1, log=True),
        'solver': trial.suggest_categorical('solver', ['adam', 'sgd']),
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh'])
    }

    aucs = []
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = MLPClassifier(
            **params,
            max_iter=300,
            random_state=42
        )

        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        aucs.append(auc)

    return np.mean(aucs)

# Crear estudio y optimizar
study = optuna.create_study(direction='maximize', study_name='MLP_AUC_Optimization')
study.optimize(objective, n_trials=150, n_jobs=1)  

# Mostrar mejores resultados
print("Mejor AUC-ROC:", round(study.best_value, 4))
print("Mejores hiperparámetros encontrados:")
print(study.best_params)

 Mejor AUC-ROC: 0.7108
 Mejores hiperparámetros encontrados:
{'hidden_layer_sizes': (50,), 'alpha': 1.1657970122657613e-05, 'learning_rate_init': 0.0005703910810991549, 'solver': 'adam', 'activation': 'relu'}


In [ ]:
# 1. Modelo con los mejores hiperparámetros encontrados por Optuna
best_mlp = MLPClassifier(
    hidden_layer_sizes=(50,),
    alpha=1.1657970122657613e-05,
    learning_rate_init=0.0005703910810991549,
    solver='adam',
    activation='relu',
    max_iter=300,
    random_state=42
)

# 2. Validación cruzada (probabilidades)
y_proba = cross_val_predict(best_mlp, X, y, cv=5, method='predict_proba')
y_pred = (y_proba[:, 1] >= 0.5).astype(int)

# 3. Métricas
print("MÉTRICAS DEL MLP OPTIMIZADO (Optuna final):")
print("Accuracy:", round(accuracy_score(y, y_pred), 3))
print("Balanced Accuracy:", round(balanced_accuracy_score(y, y_pred), 3))
print("Precision:", round(precision_score(y, y_pred), 3))
print("Recall:", round(recall_score(y, y_pred), 3))
print("F1-Score:", round(f1_score(y, y_pred), 3))
print("AUC-ROC:", round(roc_auc_score(y, y_proba[:, 1]), 3))
print("Log Loss:", round(log_loss(y, y_proba[:, 1]), 3))

# 4. Matriz de confusión
print("\nMatriz de Confusión:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))

### Comparativa completa de modelos (Gradient Boosting y MLP)

| Métrica               | GB    Optuna | GB    RandomSearch | GB     Original | MLP     Optuna | MLP     RandomSearch | MLP     Original |
|-----------------------|--------------|---------------------|----------------|----------------|----------------------|------------------|
| **Accuracy**          | 0.643        | **0.718**           | 0.717          | **0.718**      | 0.714                | 0.714            |
| **Balanced Accuracy** | **0.658**    | 0.530               | 0.553          | 0.557          | 0.556                | 0.556            |
| **Precision**         | **0.835**    | 0.727               | 0.738          | 0.739          | **0.740**            | 0.739            |
| **Recall**            | 0.623        | **0.969**           | **0.936**      | **0.935**      | 0.926                | 0.925            |
| **F1-Score**          | 0.714        | **0.831**           | **0.825**      | **0.826**      | 0.822                | 0.822            |
| **AUC-ROC**           | **0.713**    | 0.701               | 0.699          | 0.700          | 0.700                | 0.699            |
| **Log Loss**          | 0.613        | **0.547**           | **0.547**      | **0.548**      | 0.551                | 0.551            |

---

###  Conclusiones

- **Gradient Boosting**:
  - Optuna logra el **mejor AUC-ROC** y balance entre clases.
  -  RandomSearch logra el **mejor Accuracy, Recall y F1**, pero es más sesgado.
  -  Original ya era sólido, pero ambos métodos  mejoran alguna métrica.  

    
---
- **MLPClassifier**:
  - Las diferencias entre versiones son **mínimas**.
  -  Optuna mejora ligeramente en F1 y AUC-ROC.
  -  RandomSearch tiene mejor Precision.
  -  Original ya tenía un rendimiento alto, las mejoras son sutiles.

---



# Curva de precision y recall en función del umbral Gradien Boosting y MLP 

In [ ]:
# Modelos optimizados
best_gb = GradientBoostingClassifier(
    learning_rate=0.0223,
    n_estimators=484,
    max_depth=3,
    subsample=0.6375,
    random_state=42
)

best_mlp = MLPClassifier(
    hidden_layer_sizes=(50,),
    alpha=1.1658e-05,
    learning_rate_init=0.00057,
    solver='adam',
    activation='relu',
    max_iter=300,
    random_state=42
)

# Cross-validation para obtener probabilidades
y_scores_gb = cross_val_predict(best_gb, X, y, cv=5, method='predict_proba')[:, 1]
y_scores_mlp = cross_val_predict(best_mlp, X, y, cv=5, method='predict_proba')[:, 1]

# Calcular curvas
precision_gb, recall_gb, thresholds_gb = precision_recall_curve(y, y_scores_gb)
precision_mlp, recall_mlp, thresholds_mlp = precision_recall_curve(y, y_scores_mlp)

# Grafico
plt.figure(figsize=(10, 6))
plt.plot(thresholds_gb, precision_gb[:-1], label='Precision GB')
plt.plot(thresholds_gb, recall_gb[:-1], label='Recall GB')

plt.plot(thresholds_mlp, precision_mlp[:-1], label='Precision MLP', linestyle='--')
plt.plot(thresholds_mlp, recall_mlp[:-1], label='Recall MLP', linestyle='--')

plt.xlabel('Umbral de decisión')
plt.ylabel('Valor')
plt.title('Precision y Recall según umbral de decisión')
plt.legend()
plt.grid(True)
plt.show()

##  Análisis de Precision y Recall según el Umbral de Decisión

La gráfica muestra cómo varían la **precisión** y el **recall** de los modelos optimizados (**Gradient Boosting** y **MLPClassifier**) en función del umbral de decisión aplicado a las probabilidades predichas.

###  Objetivo de esta exploración

- Evaluar cómo cambia el comportamiento del modelo cuando ajustamos el umbral de clasificación (por defecto es 0.5).
- Identificar si hay un punto de equilibrio entre **precision** y **recall**.
- Decidir si conviene modificar el umbral para mejorar el rendimiento según el objetivo del modelo.

---

##  Observaciones clave de la gráfica

- Cuando el **umbral es bajo** (cerca de 0.0), ambos modelos presentan:
  - **Recall alto** (cerca de 1.0): detectan casi todos los positivos.
  - **Precision baja**: hay muchos falsos positivos.

- Cuando el **umbral sube** hacia 1.0:
  - La **precisión aumenta**: los positivos predichos son más confiables.
  - El **recall cae**: se pierden muchas verdaderas clases positivas.



---

##  Conclusiones 

- La gráfica permite **ajustar el umbral de decisión de forma informada**: si el objetivo es **detectar con más seguridad cuándo gana el favorito** (más recall), se puede elegir un umbral bajo (ej. 0.4).
- Si en cambio se quiere **ser más preciso** al decir que el favorito gana (menos falsos positivos), se puede optar por umbrales más altos (ej. 0.6).


Segun el contexto se debe elegir un umbral u otro:
- **Si es más grave predecir que ganará el favorito cuando no lo hace** → prioriza **precision**.
- **Si es más importante detectar casi todas las veces que gana el favorito** → prioriza **recall**.
